<a href="https://colab.research.google.com/github/RichardTesla/datal/blob/main/Comparative_Analysis_of_Mortgage_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# libraries
import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px

# Format money columns to two decimal places in pandas
pd.options.display.float_format = '{:6.2f}'.format

# Step 3:
Derivation of the monthly mortgage payment formula:

Definitions:

n: number of months of the mortgage payment

i: monthly interest rate

p: principal (at the beginning)

m: monthly mortgage payment

Concept:

Step 1: $h = p*i$ is the interest paid in the current month.

Step 2: $c = m - h$ is the difference between monthly payment and the monthly interest payment . This is basically the amount of principal paid for the month.

Step 3: $q = p - c$ is the new principal for the loan

Step 4: set $p=q$ and repeat step 1.


Calculating required variables for the first month, we get:

 $h = p*i$

 $c = m - p*i$

 $q = p - ( m - p*i) = p*(1 + i) - m$


For the second month:

 $h = (p*( 1 + i ) - m)*i$

 $c = m - (p*i*( i + 1 ) - m*i)$

 $q = p*( 1 + i ) - m - (m - p*i*( i + 1 ) - m*i) = p*( 1 + i )^{2} - m*( 1 + i ) - m$


Similarly, after third month,

$q = p*( 1 + i )^{3} - m*( 1 + i )^{2} - m*( 1 + i ) - m$

$m*( 1 + i )^{2}  + m*( 1 + i ) + m$ is a  GP (geometric progression)  with first term = m and common ratio = (1+i)

Sum of n terms of this progression =  $m*[ (1 - ( 1 + i )^{n} ) / ( 1- ( 1 + i ) ]$  

At the end of all the payment periods,  q should be 0 since no principal should be left to be paid.

Setting q = 0, we get:

   $0 = p*( 1 + i )^n - m*[ (1 - ( 1 + i )^{n} ) / ( 1- ( 1 + i ) ]$

 Solving for m, we get:

  $m = p * [ (i*( i + 1 )^{n}) / ( ( 1 + i )^{n} - 1) ]$




In [ ]:
def create_amortization_schedule(Principal, InterestRate, months, years):

  # prepare a DataFrame
  payments = pd.DataFrame(columns = ['Period', 'BeginBalance', 'Payment',  'InterestPaid','PrincipalPaid', 'TotalInterestPaid','TotalPrincipalPaid', 'UnpaidBalance'])

  # calculate total periodic payment amount once more
  Amount = Principal * (InterestRate / months) * np.power((1+(InterestRate/months)), (years*months)) / (np.power((1+(InterestRate/months)), (years*months))-1)

  # initialize first row to 0. We will need to reference
  # the value of principal to subtract in the first iteration
  # that amount should be 0 since we want to calculate interest
  # on the full principal during the first month
  payments =  payments.append(
      {'Period': 0,
       'BeginBalance': Principal,
       'Payment': 0,
       'PrincipalPaid': 0,
       'InterestPaid': 0,
       'TotalPrincipalPaid': 0,
       'TotalInterestPaid': 0,
       'UnpaidBalance': Principal},
       ignore_index = True)

  # iterate through each period and find that periods components
  # there are t x n periods (e.g. months * years). We will add a
  # plus 1 since for loops don't include the last number to iterate
  # through

  for i in range(1,months*years+1):
    Period = i

    # calculate component values for interest and princpal
    InterestPaid = (InterestRate / months) * (Principal - payments['TotalPrincipalPaid'][i-1])
    PrincipalPaid = Amount-InterestPaid

    # calcualte total values for interest paid and principal paid
    TotalPrincipalPaid = payments['TotalPrincipalPaid'].iloc[i-1] + PrincipalPaid
    TotalInterestPaid = payments['TotalInterestPaid'].iloc[i-1] + InterestPaid

    # calculate principal's begin, unpaid balance and Payment
    UnpaidBalance = payments['UnpaidBalance'].iloc[i-1] - PrincipalPaid
    BeginBalance = payments['UnpaidBalance'].iloc[i-1]

    # Payment = {'Payment': payments}
    # store the results
    payments =  payments.append({'Period': Period,
                                 'BeginBalance': BeginBalance,
                                 'Payment': Amount,
                                 'PrincipalPaid': PrincipalPaid,
                                 'InterestPaid': InterestPaid,
                                 'TotalPrincipalPaid': TotalPrincipalPaid,
                                 'TotalInterestPaid': TotalInterestPaid,
                                 'UnpaidBalance': UnpaidBalance},
                              ignore_index = True)


  # change our period to an integer instead of a float
  payments['Period'] = payments['Period'].astype('int')

  # monthly mortgage payment inputs
  monthlyInterestRate = InterestRate/months
  t = months*years

  # monthly mortgage payment
  # m = (Principal*(intst*(intst+1)**t))/((intst+1)**t - 1)

  print(f"1. Mortgage Outputs")
  print(f"==="*50)
  print(f"The monthly interest rate is {round(monthlyInterestRate*100,2)}%, principal amount is ${Principal}, number of monthly payments is {months*years} days and monthly payment is ${round(Amount,2)}")
  print(f"---"*50)
  print(f"The total amount paid in full is ${round(Principal+payments.TotalInterestPaid.iloc[-1],2)}.")
  print(f"---"*50)
  print(f"Total interest paid is ${round(payments.TotalInterestPaid.iloc[-1],2)}.")
  print(f"==="*50)
  print(f"\n")
  print(f"2. Amortization Schedule")
  print(f"="*50)
  # Show last 5 rows in the amortization table
  return payments

def loan_amort_viz(data, title):
  # Plotting InterestPaid and PrincipalPaid components within one plot
  fig = px.line(
      data_frame=data.iloc[1:],
      x="Period",
      y=["PrincipalPaid", "InterestPaid"],
      title=title,
      labels={"variable": "Components", "PrincipalPaid": "Principal paid"}
      )

  # Update plot layout
  fig.update_layout(
      height=400,
      width=800,
      xaxis_title="Period",
      yaxis_title="Principal/Interest Paid",

  )
  return fig.show()


### 3.1 Mortgage 1: A 30-year fixed rate at 4%. The loan will amortize over 30 years

---



In [ ]:
df1=create_amortization_schedule(Principal=1000000, InterestRate=0.04, months=12, years=30)
df1

1. Mortgage Outputs
The monthly interest rate is 0.33%, principal amount is $1000000, number of monthly payments is 360 days and monthly payment is $4774.15
------------------------------------------------------------------------------------------------------------------------------------------------------
The total amount paid in full is $1718695.06.
------------------------------------------------------------------------------------------------------------------------------------------------------
Total interest paid is $718695.06.


2. Amortization Schedule


,Period,BeginBalance,Payment,InterestPaid,PrincipalPaid,TotalInterestPaid,TotalPrincipalPaid,UnpaidBalance
0,0,1000000,0,0,0,0,0,1000000
1,1,1000000.00,4774.15,3333.33,1440.82,3333.33,1440.82,998559.18
2,2,998559.18,4774.15,3328.53,1445.62,6661.86,2886.44,997113.56
3,3,997113.56,4774.15,3323.71,1450.44,9985.58,4336.88,995663.12
4,4,995663.12,4774.15,3318.88,1455.28,13304.45,5792.16,994207.84
...,...,...,...,...,...,...,...,...
356,356,23633.90,4774.15,78.78,4695.37,718536.98,981061.47,18938.53
357,357,18938.53,4774.15,63.13,4711.02,718600.11,985772.50,14227.50
358,358,14227.50,4774.15,47.43,4726.73,718647.53,990499.22,9500.78
359,359,9500.78,4774.15,31.67,4742.48,718679.20,995241.71,4758.29


In [ ]:
loan_amort_viz(data=df1, title="<b> Mortgage 1: 30-year Fixed Rate Mortgage @4% Interest Rate Amoritization Components</b>")

### 3.2 Mortgage 2: A 20-year fixed rate at 2.5%. The loan will amortize over 20 years


In [ ]:
df2=create_amortization_schedule(Principal=1000000, InterestRate=0.025, months=12, years=20)
df2

1. Mortgage Outputs
The monthly interest rate is 0.21%, principal amount is $1000000, number of monthly payments is 240 days and monthly payment is $5299.03
------------------------------------------------------------------------------------------------------------------------------------------------------
The total amount paid in full is $1271766.94.
------------------------------------------------------------------------------------------------------------------------------------------------------
Total interest paid is $271766.94.


2. Amortization Schedule


,Period,BeginBalance,Payment,InterestPaid,PrincipalPaid,TotalInterestPaid,TotalPrincipalPaid,UnpaidBalance
0,0,1000000,0,0,0,0,0,1000000
1,1,1000000.00,5299.03,2083.33,3215.70,2083.33,3215.70,996784.30
2,2,996784.30,5299.03,2076.63,3222.39,4159.97,6438.09,993561.91
3,3,993561.91,5299.03,2069.92,3229.11,6229.89,9667.20,990332.80
4,4,990332.80,5299.03,2063.19,3235.84,8293.08,12903.03,987096.97
...,...,...,...,...,...,...,...,...
236,236,26330.35,5299.03,54.85,5244.17,271657.01,978913.82,21086.18
237,237,21086.18,5299.03,43.93,5255.10,271700.93,984168.92,15831.08
238,238,15831.08,5299.03,32.98,5266.05,271733.92,989434.97,10565.03
239,239,10565.03,5299.03,22.01,5277.02,271755.93,994711.99,5288.01


In [ ]:
loan_amort_viz(data=df2, title="<b> Mortgage 2: 20-year Fixed Rate Mortgage @2.5% Interest rate Amoritization Components</b>")

### 3.3 Mortgage 3: A 7-1 Adjustable Rate Mortgage (ARM) that varies according to rates. The loan will amortize over 30 years.



A 7/1 ARM is a mortgage that has a fixed interest rate in the beginning, then it switches to an adjustable or variable one.

The 7 in 7/1 means the initial fixed period is of 7 years. After that, the interest rate adjusts once yearly based on some index stated in the contract.
             
Principal at the beginning, p = $ 1,000,000 .

As per the question, we are using simulated rates which are provided on a weekly basis.

The simulated rates have been provided till June 9, 2022.

Hence, our last payment should be before June 9, 2022.
              
We are assuming that the last payment is on June 1, 2022.  

Hence, in the amortization schedule, we are starting the payments from July 1, 1992.

First payment on July 1, 1992 is pertaining to last month of June 1992.
              
For first 7 years, the annual interest rate is fixed at 8.59%.
              
After 7 years, annual interest rate is taken as the value of the first available interest rate in June of the corresponding year. Also, the updated interest rate is applicable from month of June of the corresponding year (with first payment with updated interest rate on July 1st of the corresponding year).

With these assumptions, Total interest payment = $ 1,344,744.15


In [ ]:
def monthly_payment( interest_rate, principal, number_of_months):
  return principal * (interest_rate / 12) * np.power((1+(interest_rate/12)), (number_of_months)) / (np.power((1+(interest_rate/12)), (number_of_months))-1)


def arm_amortization_schedule( principal, arm_rates, months, years):

  payments = pd.DataFrame(columns = ['Period', 'BeginBalance', 'InterestRate','Payment',  'InterestPaid','PrincipalPaid', 'TotalInterestPaid','TotalPrincipalPaid', 'UnpaidBalance'])

  # initialize first row to 0. We will need to reference
  # the value of principal to subtract in the first iteration
  # that amount should be 0 since we want to calculate interest
  # on the full principal during the first month
  payments =  payments.append(
      {'Period': 0,
       'BeginBalance': principal,
       'InterestRate': str(arm_rates[0]*100)+'%',
       'Payment': 0,
       'PrincipalPaid': 0,
       'InterestPaid': 0,
       'TotalPrincipalPaid': 0,
       'TotalInterestPaid': 0,
       'UnpaidBalance': principal},
       ignore_index = True)

  UnpaidBalance = principal

  for i in range(1,months*years+1):
    Period = i

    # calculate component values for interest and princpal
    Amount = monthly_payment( arm_rates[i-1], UnpaidBalance,  months*years + 1 -i )
    InterestPaid = (arm_rates[i-1] / months) * (principal - payments['TotalPrincipalPaid'][i-1])
    PrincipalPaid = Amount-InterestPaid

    # calcualte total values for interest paid and principal paid
    TotalPrincipalPaid = payments['TotalPrincipalPaid'].iloc[i-1] + PrincipalPaid
    TotalInterestPaid = payments['TotalInterestPaid'].iloc[i-1] + InterestPaid

    # calculate principal's begin, unpaid balance and Payment
    UnpaidBalance = payments['UnpaidBalance'].iloc[i-1] - PrincipalPaid
    BeginBalance = payments['UnpaidBalance'].iloc[i-1]

    # Payment = {'Payment': payments}
    # store the results
    payments =  payments.append({'Period': Period,
                                 'BeginBalance': BeginBalance,
                                 'InterestRate': str(arm_rates[i-1]*100)+'%',
                                 'Payment': Amount,
                                 'PrincipalPaid': PrincipalPaid,
                                 'InterestPaid': InterestPaid,
                                 'TotalPrincipalPaid': TotalPrincipalPaid,
                                 'TotalInterestPaid': TotalInterestPaid,
                                 'UnpaidBalance': UnpaidBalance},
                              ignore_index = True)
  # change our period to an integer instead of a float
  payments['Period'] = payments['Period'].astype('int')

  t = months*years

  # monthly mortgage payment
  # m = (Principal*(intst*(intst+1)**t))/((intst+1)**t - 1)

  print(f"1. Mortgage Outputs")
  print(f"==="*50)
  print(f"Principal amount is ${principal}, number of monthly payments is {months*years}" )
  print(f"---"*50)
  print(f"The total amount paid in full is ${round(principal+payments.TotalInterestPaid.iloc[-1],2)}.")
  print(f"---"*50)
  print(f"Total interest paid is ${round(payments.TotalInterestPaid.iloc[-1],2)}.")
  print(f"==="*50)
  print(f"\n")
  print(f"2. Amortization Schedule")
  print(f"="*50)
  # Show last 5 rows in the amortization table
  return payments


def loan_amort_viz(data, title):
  # Plotting InterestPaid and PrincipalPaid components within one plot
  fig = px.line(
      data_frame=data.iloc[1:],
      x="Period",
      y=["PrincipalPaid", "InterestPaid"],
      title=title,
      labels={"variable": "Components", "PrincipalPaid": "Principal paid"}
      )

  # Update plot layout
  fig.update_layout(
      height=400,
      width=800,
      xaxis_title="Period",
      yaxis_title="Principal/Interest Paid",

  )
  return fig.show()


In [ ]:
arm_rates1 = [8.59]*84
arm_rates2 = [7.41, 8.54, 7.24, 6.71, 5.26, 6.28, 5.62, 6.67, 6.53, 6.09, 5.29, 4.79, 4.55, 3.67, 3.91, 4.14, 3.87, 3.66, 3.94, 4.54, 3.82, 3.18, 2.99]

arm_rate = arm_rates1

for rate in arm_rates2:
  for j in range(0,12):
    arm_rate.append(rate)

arm_rate = [rate*0.01 for rate in arm_rate]

df3 = arm_amortization_schedule( principal=1000000, arm_rates= arm_rate, months=12, years=30)
df3

1. Mortgage Outputs
Principal amount is $1000000, number of monthly payments is 360
------------------------------------------------------------------------------------------------------------------------------------------------------
The total amount paid in full is $2344744.15.
------------------------------------------------------------------------------------------------------------------------------------------------------
Total interest paid is $1344744.15.


2. Amortization Schedule


,Period,BeginBalance,InterestRate,Payment,InterestPaid,PrincipalPaid,TotalInterestPaid,TotalPrincipalPaid,UnpaidBalance
0,0,1000000,8.59%,0,0,0,0,0,1000000
1,1,1000000,8.59%,7753.01,7158.33,594.68,7158.33,594.68,999405.32
2,2,999405.32,8.59%,7753.01,7154.08,598.93,14312.41,1193.61,998806.39
3,3,998806.39,8.59%,7753.01,7149.79,603.22,21462.20,1796.83,998203.17
4,4,998203.17,8.59%,7753.01,7145.47,607.54,28607.67,2404.37,997595.63
...,...,...,...,...,...,...,...,...,...
356,356,28004.92,2.99%,5642.92,69.78,5573.14,1344604.24,977568.22,22431.78
357,357,22431.78,2.99%,5642.92,55.89,5587.03,1344660.13,983155.25,16844.75
358,358,16844.75,2.99%,5642.92,41.97,5600.95,1344702.11,988756.20,11243.80
359,359,11243.80,2.99%,5642.92,28.02,5614.91,1344730.12,994371.10,5628.90


In [ ]:
loan_amort_viz(data=df3, title="<b> Mortgage 3:  7-1 Adjustable Rate Mortgage (ARM) Amortization Components</b>")